# Luke

In [ ]:
import pandas as pd
df = pd.read_csv("../data/raw/US_Gun_Violence_Data.csv")

      incident_id    incident_date         state  city_or_county  \
0         2015849      May 26 2021      Illinois         Chicago   
1         2015730      May 26 2021    California        San Jose   
2         2014343      May 24 2021      Michigan         Inkster   
3         2014381      May 24 2021          Ohio  West Jefferson   
4         2012971      May 23 2021      Illinois         Chicago   
...           ...              ...           ...             ...   
2925        95550  January 12 2014       Alabama      Huntsville   
2926        95146  January 11 2014   Mississippi         Jackson   
2927        94514   January 5 2014  Pennsylvania            Erie   
2928        92704   January 3 2014      New York          Queens   
2929        92194   January 1 2014      Virginia         Norfolk   

                                            address  killed  injured  
0                     7200 block of S Princeton Ave       0        4  
1                                 101 W Y

In [ ]:
df.plot(x='incident_date', y=['killed','injured'])

Looking at a plot of killed and injured over incident_date shows multiple outliers with killed having a max of 59 and injured haveing a max of 441. These will have to be looked at and possibly removed. incident_date may also need to be simplified and split into years or months.

In [ ]:
df.plot(kind='scatter', x='state', y='killed')

In [ ]:
import pandas as pd
df = pd.read_csv("../data/raw/US_Gun_Violence_Data.csv")

df.nunique(axis=0)
df.describe().apply(lambda s: s.apply(lambda x: format(x, 'f')))


def clean_month(row):
    
    months = ['January','February','March','April','May','June','July','August','September','October','November','December']
    month = [m for m in months if(m in row.incident_date)]
    return month[0]
    # Clean dataframe
    
def clean_year(row):
    
    years = ['2014','2015','2016','2017','2018','2019','2020','2021','2022'] 
    year = [y for y in years if(y in row.incident_date)]
    return int(year[0])
    # Clean dataframe
   
    
def clean_df(playlist):
    df_cleaned = df.copy()
    
    #split incident_date into two new columns named 'month' and 'year'
    #month column for more general info and for possible research question on violence per month
    #year column required for possible questions and to remove incomplete years
    df_cleaned['month'] = df_cleaned.apply(lambda row: clean_month(row), axis=1)
    df_cleaned['year'] = df_cleaned.apply(lambda row: clean_year(row), axis=1)
    df_cleaned = df_cleaned.drop(['incident_id','address','city_or_county','incident_date'], axis=1)
    
    #remove outliers
    df_cleaned = df_cleaned[df_cleaned['killed'] < 50]
    df_cleaned = df_cleaned[df_cleaned['injured'] < 50]
    df_cleaned = df_cleaned[df_cleaned['year'] < 2021]

    #rearrange columns
    df_cleaned = df_cleaned[['year','month','state','killed','injured']]
    return df_cleaned# Get df with reclassfied 'condition' column

df_cleaned = clean_df(df)


## Research question/interests

Briefly describe your research question or interests here.